# Model I/O

Format ➡️ Model ➡️ Parse 

In [15]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage

In [7]:
llm = OpenAI()
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

text = "Give me a quote of the day! I need some inspiration."

llm_response = llm.invoke(text)

msgs = [
    HumanMessage(content=text)
]
chat_response = chat_model.invoke(msgs)

print(llm_response)
print(chat_response)



"Believe you can and you're halfway there." - Theodore Roosevelt
content='"The only way to do great work is to love what you do." - Steve Jobs' response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 20, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-c174ba51-8b82-41f8-b789-14ac003cc995-0'


In [8]:
print(type(llm_response))
print(type(chat_response))

<class 'str'>
<class 'langchain_core.messages.ai.AIMessage'>


## Prompt Templates

In [14]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

In [9]:
prompt = PromptTemplate.from_template("Give me a quote by the philosopher {philosopher}.")
prompt.format(philosopher= "Aristotle")

'Give me a quote by the philosopher Aristotle.'

In [13]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert language translator. Translate the following {input_lang} text to {output_lang}."),
    ("user", "{text}")
])

chat_prompt.format_messages(input_lang="English", output_lang="French", text="Hello, how are you?")

[SystemMessage(content='You are an expert language translator. Translate the following English text to French.'),
 HumanMessage(content='Hello, how are you?')]

## Output parsers

In [16]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import SimpleJsonOutputParser

In [17]:
list_parser = CommaSeparatedListOutputParser()
list_parser.parse("apple, banana, cherry")

['apple', 'banana', 'cherry']

In [18]:
json_parser = SimpleJsonOutputParser()
json_parser.parse('{"name": "Nikhil", "age": 25, "city": "New York"}')

{'name': 'Nikhil', 'age': 25, 'city': 'New York'}

In [21]:
print(list_parser.get_format_instructions())
print(json_parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
Return a JSON object.


# Composing with LCEL (LangChain Expression Language)

In [22]:
template = "Generate a list of 5 quotes by the philosopher {philosopher}.\n\n{format_instructions}"
chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=list_parser.get_format_instructions())

chain = chat_prompt | chat_model | list_parser
response = chain.invoke({"philosopher": "Aristotle"})

In [24]:
response

['1. "The more you know',
 'the more you realize you don’t know."\n2. "We are what we repeatedly do. Excellence',
 'then',
 'is not an act',
 'but a habit."\n3. "Happiness depends upon ourselves."\n4. "It is the mark of an educated mind to be able to entertain a thought without accepting it."\n5. "The whole is greater than the sum of its parts."']